In [ ]:
!pip install tensorflow==2.3.1 gym keras-rl2 gym[atari] pip install gym[accept-rom-license]

imports

In [ ]:
import gym 
import random
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

CRIANDO AMBIENTE NO GYM

In [ ]:
env = gym.make('SpaceInvaders-v0')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

MODELO ADOTADO PARA TREINAR O AGENTE

In [ ]:
def modelo(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

AGENTE

In [ ]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

TREINO DO MODELO E SUAS PONTUACOES

In [ ]:
model = modelo(height, width, channels, actions)

In [ ]:
deepQLearning = build_agent(model, actions)
deepQLearning.compile(Adam(lr=1e-4))

In [ ]:
deepQLearning.fit(env, nb_steps=1000, visualize=False, verbose=2)

In [ ]:
scores = deepQLearning.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))

In [ ]:
deepQLearning.save_weights('C:/Users/biela/dqn_weights.h5f')

In [ ]:
del model, deepQLearning

In [ ]:
deepQLearning.load_weights('C:/Users/biela/dqn_weights.h5f')